In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI
import tiktoken
from requests_ratelimiter import LimiterSession
from token_limiter import TokenLimiter

In [ ]:
load_dotenv()

True

In [3]:
def read_data(filepath):
    return pd.read_csv(filepath, on_bad_lines='skip')

In [4]:
def clean_genre(genre_str):
    genre_str = genre_str.replace("[", "")
    genre_str = genre_str.replace("]", "")
    genre_str = genre_str.replace("'", "")
    return genre_str

In [5]:
def clean_author(author_str):
    pos = author_str.find("(")
    return author_str[:pos]

In [6]:
filepath = r"/home/azureuser/notebooks/mlops_book_recommendation_system/data/books_1.Best_Books_Ever.csv"
df = read_data(filepath)

In [7]:
df

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,01/28/13,[],2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,10/05/05,"['Georgia Peach Book Award (2007)', 'Buxtehude...",4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52473,11492014-fractured,Fractured,Fateful #2,Cheri Schmidt (Goodreads Author),4.00,The Fateful Trilogy continues with Fractured. ...,English,2940012616562,"['Vampires', 'Paranormal', 'Young Adult', 'Rom...",[],...,NaN,[],871,"['311', '310', '197', '42', '11']",94.0,[],https://i.gr-assets.com/images/S/compressed.ph...,0,1,NaN
52474,11836711-anasazi,Anasazi,Sense of Truth #2,Emma Michaels,4.19,"'Anasazi', sequel to 'The Thirteenth Chime' by...",English,9999999999999,"['Mystery', 'Young Adult']",[],...,August 3rd 2011,[],37,"['16', '14', '5', '2', '0']",95.0,[],https://i.gr-assets.com/images/S/compressed.ph...,0,1,NaN
52475,10815662-marked,Marked,Soul Guardians #1,Kim Richardson (Goodreads Author),3.70,--READERS FAVORITE AWARDS WINNER 2011--Sixteen...,English,9781461017097,"['Fantasy', 'Young Adult', 'Paranormal', 'Ange...",[],...,March 15th 2011,"[""Readers' Favorite Book Award (2011)""]",6674,"['2109', '1868', '1660', '647', '390']",84.0,[],https://i.gr-assets.com/images/S/compressed.ph...,0,1,7.37
52476,11330278-wayward-son,Wayward Son,NaN,"Tom Pollack (Goodreads Author), John Loftus (G...",3.85,A POWERFUL TREMOR UNEARTHS AN ANCIENT SECRETBu...,English

In [8]:
feature_df = df[["title", "author", "description", "genres", "likedPercent", "numRatings"]]

In [9]:
feature_df = feature_df[feature_df['description'].notna()]
feature_df['genres'] = feature_df['genres'].apply(lambda x: clean_genre(x))
feature_df = feature_df[feature_df['numRatings'] > 500]
feature_df['author'] = feature_df['author'].apply(lambda x: clean_author(x))

In [10]:
feature_df

,title,author,description,genres,likedPercent,numRatings
0,The Hunger Games,Suzanne Collin,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"Young Adult, Fiction, Dystopia, Fantasy, Scien...",96.0,6376780
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré",There is a door at the end of a silent corrido...,"Fantasy, Young Adult, Fiction, Magic, Children...",98.0,2507623
2,To Kill a Mockingbird,Harper Le,The unforgettable novel of a childhood in a sl...,"Classics, Fiction, Historical Fiction, School,...",95.0,4501075
3,Pride and Prejudice,"Jane Austen, Anna Quindlen",Alternate cover edition of ISBN 9780679783268S...,"Classics, Fiction, Romance, Historical Fiction...",94.0,2998241
4,Twilight,Stephenie Meye,About three things I was absolutely positive.\...,"Young Adult, Fantasy, Romance, Vampires, Ficti...",78.0,4964519
...,...,...,...,...,...,...
52470,Attracted to Fire,DiAnn Mills,Special Agent Meghan Connors' dream of one day...,"Christian Fiction, Christian, Suspense, Romanc...",95.0,2143
52471,Elemental,Kim Richardson,When seventeen-year-old Kara Nightingale is su...,"Fantasy, Young Adult, Angels, Romance, Paranor...",94.0,1947
52472,Unbelievable,Sherry Gammon,Lilah Lopez Dreser's in town to take care of u...,"Romance, Young Adult, Contemporary, Contempora...",94.0,1028
52473,Fractured,Cheri Schmidt,The Fateful Trilogy continues with Fractured. ...,"Vampires, Paranormal, Young Adult, Romance, Fa...",94.0,871


In [11]:
def count_tokens(text: str | list):
    encoding = tiktoken.encoding_for_model('text-embedding-3-small')
    num_tokens = 0

    if type(text) == str:
        num_tokens = len(encoding.encode(text))
    elif type(text) == list:
        num_tokens = sum([count_tokens(txt) for txt in text])
    else:
        raise ValueError(f"count_tokens does not support input of type {type(text)}. Please ensure your input is a string value or a list of strings.")
        
    return num_tokens 

In [12]:
def check_tokens(text):
    return count_tokens(text) <= 8191

In [13]:
categorical = "Title: " + feature_df.title + " Author: " + feature_df.author + " Description: " + feature_df.description + " Genres: " + feature_df.genres 

In [14]:
categorical = categorical[categorical.apply(check_tokens)]

In [15]:
numerical = pd.DataFrame(feature_df[['likedPercent', 'numRatings']]) 

In [16]:
# scale the numerical features
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(numerical)

In [17]:
def _get_embeddings(texts: str | list, limiter: TokenLimiter, session: LimiterSession, model="text-embedding-3-small"):

    if type(texts) != str and type(texts) != list:
        raise ValueError("Must pass a string or a list to create embeddings")

    with session:
        client = AzureOpenAI(
            api_key = os.getenv("AZURE_OPENAI_API_KEY"),
            api_version = "2024-10-21",
            azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
        )

        token_count = count_tokens(texts)
        limiter.wait_for_slot(token_count)

        response = client.embeddings.create(
            model = model,
            input = texts
        )

        return response.data

In [18]:
def get_embeddings(data: str | list, step=10) -> list:
    embeddings = [] 

    token_limiter = TokenLimiter()
    limiter_session = LimiterSession(per_minute=700) # Change to variable

    if type(data) == list:
        for i in range(0, len(data), step):
            batch = _get_embeddings(data[i:i+step], token_limiter, limiter_session)
            embeddings += [item.embedding for item in batch]
    elif type(data) == str:
        return _get_embeddings(data, token_limiter, limiter_session).data[0].embedding
    else:
        raise ValueError(f"Data of type {type(data)} is not supported. Please use string or list.")

    return embeddings

In [19]:
categorical_list = categorical.values.tolist()

In [20]:
embeddings = get_embeddings(categorical_list)

In [22]:
embeddings_df = pd.DataFrame(embeddings)

In [25]:
embeddings_df.to_csv("data/open_ai_embeddings.csv")